In [ ]:
!mkdir elct

In [ ]:
!cat /content/sonali/Course 2 Mod 2 Topic list and Reading list (2).docx

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `cat /content/sonali/Course 2 Mod 2 Topic list and Reading list (2).docx'


In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-incf_sh5
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-incf_sh5
  Resolved https://github.com/deepset-ai/haystack.git to commit 255072d8d548a19a1678ddc46b41d41cf5d09bc4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2
import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [ ]:
doc_dir = "/content/elct"
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO:haystack.utils.preprocessing:Converting /content/elct/Course 1 Mod 8 Topic 4 Talking Points Documentation the progress note is the bill (1).docx


In [ ]:
print(docs[:3])
document_store.write_documents(docs)

[<Document: {'content': 'Coding; the Language of Medicine\nDocumentation – the progress note is the bill \nThis topic was not intended to be a rehash of the E/M topic \nWhat part does the progress note play in medicine, in the medical record? What is meant by “the progress note is the bill?”\nDoes the progress note differ from one payment scheme to another? For example, are there different elements needed for progress notes in a fee for service environment versus a Medicare Risk Adjustment or Veterans environment? Should we document “payer-blind” and what is meant by that?\nWhat are the important elements that must be in the note in order to be able to tell the story of the encounter(s)?\nHistory of present illness (medical history, family history, social history)\nWhat are the Do’s and Don’ts of documentation in a progress note?\nMust not have conflicting information', 'content_type': 'text', 'score': None, 'meta': {'name': 'Course 1 Mod 8 Topic 4 Talking Points Documentation the prog

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="Chief complaint ?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

In [ ]:
from pprint import pprint

pprint(prediction['answers'])

[]


In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = ' Medicare Risk Adjustment  '
result = pipeline.run(query, params={"Retriever": {"top_k": 10}})

In [ ]:
from haystack.utils import print_documents

print_documents(result, max_text_len=100, print_name=True, print_meta=True)


Query:  Medicare Risk Adjustment  

{   'content': 'Coding; the Language of Medicine\n'
               'Documentation – the progress note is the bill \n'
               'This topic was not i...',
    'meta': {   'name': 'Course 1 Mod 8 Topic 4 Talking Points Documentation '
                        'the progress note is the bill (1).docx'},
    'name': 'Course 1 Mod 8 Topic 4 Talking Points Documentation the progress '
            'note is the bill (1).docx'}



In [ ]:
[x.to_dict() for x in result["documents"]]

[{'content': 'Coding; the Language of Medicine\nDocumentation – the progress note is the bill \nThis topic was not intended to be a rehash of the E/M topic \nWhat part does the progress note play in medicine, in the medical record? What is meant by “the progress note is the bill?”\nDoes the progress note differ from one payment scheme to another? For example, are there different elements needed for progress notes in a fee for service environment versus a Medicare Risk Adjustment or Veterans environment? Should we document “payer-blind” and what is meant by that?\nWhat are the important elements that must be in the note in order to be able to tell the story of the encounter(s)?\nHistory of present illness (medical history, family history, social history)\nWhat are the Do’s and Don’ts of documentation in a progress note?\nMust not have conflicting information',
  'content_type': 'text',
  'score': 0.5270956168590992,
  'meta': {'name': 'Course 1 Mod 8 Topic 4 Talking Points Documentation